In [1]:
# Make sure we have the new pandas 1.0 (Jan 2020) so we can use the new string dtype
! pip install pandas --upgrade

# Fuzzy text matching
! pip install "textdistance[extras]" --upgrade

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

path = '/content/drive/My Drive/active/m5362_20sp_data_warehousing/PD4SDG/PD4SDG_new/data sources'

def formatter(X):
    """Common cleaning and formatting"""
    # convert to modern dtypes like "string" introduced in Pandas 1.0 (Jan 2020)
    df = X.copy().convert_dtypes()
    
    # trim whitespace, replace missing and length 0 or 1 strings with '', standard capitalization
    f = lambda x: x.str.strip().fillna('').str.lower() \
        .str.replace(r'^.{0,1}$', '').str.replace(r'^st\.', 'saint').str.replace(r'^st', 'saint').str.replace('&', 'and') \
        .str.title()
    
    # format column names, but use lowercase (my preference)
    df.columns = f(df.columns).str.lower()
    
    # format string columns
    strings = df.select_dtypes(include=['string','object']).columns   # finds string columns
    df[strings] = df[strings].apply(f).astype('string')   # formats them
    return df

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.0.3)
Requirement already up-to-date: textdistance[extras] in /usr/local/lib/python3.6/dist-packages (4.1.5)


In [0]:
world_raw = pd.read_json(f"{path}/raw/world-cities.json").set_index('geonameid')
world = formatter(world_raw) \
    .rename(columns={'name':'city'})
world = world[cols].drop_duplicates().sort_values(['country', 'subcountry', 'city'])
world.to_csv(f"{path}/city.csv")

In [0]:
country_data_raw = pd.read_csv(f"{path}/raw/countries of the world.csv", decimal=',')
country_data = formatter(country_data_raw)
country_data.index += 1
country_data.index.rename('id', inplace=True)
country_data.to_csv(f"{path}/country_data.csv")

In [0]:
pd4sdg_raw = pd.read_excel(f"{path}/raw/PD4SDG.xlsx")
pd4sdg = formatter(pd4sdg_raw) \
    .rename(columns={'project location 1':'site', 'title':'title', 'project_idx':'un_id'})
pd4sdg['un_id'] = pd4sdg['un_id'].str.replace("'", "")

In [0]:
# Get unique un_idx list
f = lambda L: max(L, key=len)  # returns longest string in list
project = pd4sdg.groupby('un_id').agg(
    title = ('title', f),
    site = ('site', f),
    repeats = ('un_id', 'count'),
    ).reset_index()
project.index += 1
project.index.rename('id', inplace=True)
project.to_csv(f"{path}/project.csv")

In [0]:
# Get project_entity links
partners = ['lead partner'] + [f"partner {i+1}" for i in range(237)]

# Temp list to hold results
L = []
for (n, c) in enumerate(partners):
    # get partner columns, rename cols, drop rows where name is ''
    s = pd4sdg[['un_id', c]].rename(columns={c: 'name'})
    # s = s[s['name'].str.len() > 0]
    s = s[s['name'] != '']

    # record partner number on project (in case precedence matters - we don't believe it does)
    s['n'] = n

    # Append
    L.append(s)

# concat the lists stored in L
project_entity = pd.concat(L)
project_entity.index += 1
project_entity.index.rename('id', inplace=True)
project_entity.to_csv(f"{path}/project_entity.csv", index=False)

In [32]:
names = pd.Series(project_entity['name'].unique()).sort_values()
names.head(30)

1621                                             #Oneless
676                                                 (Red)
1406                                           10 By 2020
6499                                          118 Journey
3658                                     11Th Hour Racing
1320                                         2020 Or Bust
6243    23. Agricultural Leadership Foundation Of HawaiʻI
6443                           24. Blue Planet Foundation
6612         25. Coordinating Group On Alien Pest Species
6983                    28. Harold K.L. Castle Foundation
7088                       29. Hau‘Oli Mau Loa Foundation
3893                                               2Scale
5017                                           33 Buckets
7438                               35. Kamehameha Schools
5609                         350 Pacific Climate Warriors
970                                               350.Org
14                                                350Org.
7586          

In [0]:
# Get project_entity links
partners = ['lead partner'] + [f"partner {i+1}" for i in range(237)]

# Temp list to hold results
L = []
# for i, c in enumerate(partners):
#     # dict {original column name: new column name}
#     col = {'un_id':'un_id'}#, c:'name'}#, c+' type':'type', c+' city':'city', c+' country':'country'}
    
#     # get partner columns, rename cols, drop rows where name is ''
#     s = pd4sdg[col.keys()]#.rename(columns=col).astype(str)
#     # s = s[s['name'].str.len() > 1].astype(str)
#     # print(s.dtypes)
#     # record partner number on project (in case precedence matters - we don't believe it does)
#     # s['n'] = i

#     # Append
#     L.append(s)

# A = pd4sdg.set_index('un_id')
for c in partners:
    # dict {original column name: new column name}
    # col = {'un_id':'un_id', c:'name'}#, c+' type':'type', c+' city':'city', c+' country':'country'}
    
    # get partner columns, rename cols, drop rows where name is ''
    s = pd4sdg[['un_id', c]].rename(columns={c: 'name'})
    s = s[s['name'].str.len() > 0]

    # record partner number on project (in case precedence matters - we don't believe it does)
    s['n'] = i

    # Append
    L.append(s)


# concat the lists stored in L
project_entity = pd.concat(L)
print(project_entity.dtypes[:10])
project_entity.index += 1
project_entity.index.rename('id', inplace=True)
project_entity.to_csv(f"{path}/project_entity.csv")

un_id    string
name     string
n         int64
dtype: object


In [0]:
project_entity['name'].str.len().min()

2

In [0]:
import textdistance
def textdist_func(a, b):
    try:
        a[0].lower() + b[0].lower()
    except:
        return 0.0
    else:
        return textdistance.levenshtein.normalized_similarity(a, b)
textdist_ufunc = np.frompyfunc(textdist_func, 2, 1)
def textdist(A, B):
    return textdist_ufunc.outer(A, B).astype('float')


def large(X, k=2, axis=0):
    if k <= 1:
        srt = np.argmax(X, axis)  # find
        srt = np.expand_dims(srt, axis)
        X = np.take_along_axis(X, srt, axis)
    else:
        X = np.rollaxis(X, axis, 0)
        srt1 = np.argpartition(X, -k, axis=0)[-k:]
        X = np.take_along_axis(X, srt1, axis=0)
        srt2 = np.argsort(X, axis=0)[::-1]
        X = np.take_along_axis(X, srt2, axis=0)
        srt = np.take_along_axis(srt1, srt2, axis=0)
        X   = np.rollaxis(X, 0, axis+1)
        srt = np.rollaxis(srt, 0, axis+1)
    return X, srt


def text_match(orig, targ, num_matches=1):
    def f(X):
        df = pd.DataFrame(X.copy()).drop_duplicates()
        m = (df!='').any(axis=1)
        return df[m]
    orig = f(orig)
    targ = f(targ)

    all_scores = textdist(orig.to_numpy(), targ.to_numpy())
    max_scores = all_scores.max((1,3))
    best_score, srt = large(max_scores.T, k=num_matches, axis=0)

    index = targ.index.to_numpy()
    names = targ.iloc[:, 0].to_numpy()
    for i in range(num_matches):
        orig[f"score_{i+1}"] = (best_score[i]*100).round(1)
        orig[f"index_{i+1}"] =   index[srt[i]]
        orig[ f"name_{i+1}"]  =  names[srt[i]]
   
    return orig, all_scores

In [0]:
num_matches = 3
all_site = pd.read_csv(f"{path}/output/site.csv").set_index('site_id')

entity   = pd.read_csv(f"{path}/output/entity.csv").set_index('id')[['name', 'type', 'site']]
s = all_site.index > 5000
df, s = text_match(entity['site'][:5], all_site.iloc[s,3:], num_matches)
for i in range(num_matches):
    df[f"use_match_{i+1}"] = ''
df[f"use_match_{1}"] = 'x'
df['no_match_found'] = ''
entity_site_fix = entity.join(df, lsuffix='_orig').sort_values('score_1')
entity_site_fix.to_csv(f"{path}/output/entity_site_fix.csv")




# all_site = pd.read_csv(f"{path}/output/site.csv")
# entity.head()
# all_site.iloc[:,3:].head()

In [0]:
entity_site_fix.to_csv(f"{path}/output/entity_site_fix.csv")

In [0]:
entity_site_fix = entity.join(df, lsuffix='_orig').sort_values('score_1')
entity_site_fix.head()

,name,type,site_orig,site,score_1,index_1,name_1,score_2,index_2,name_2,score_3,index_3,name_3,use_match_1,use_match_2,use_match_3,no_match_found
id,,,,,,,,,,,,,,,,,
5,Community And Family Foundation,Ngo,"Accra, , Ghana","Accra, , Ghana",85.7,2306104.0,"Accra, Greater Accra, Ghana",66.7,2306079.0,"Achiaman, Greater Accra, Ghana",64.3,2306119.0,"Aburi, Eastern, Ghana",x,,,
1,"""Sail Of Hope""",Ngo,"Moscow, , Russia","Moscow, , Russia",87.5,2921.0,"Moscow, Russia",87.5,524901.0,"Moscow, Moscow, Russia",68.8,501215.0,"Rossosh’, Voronezj, Russia",x,,,
4,#NAME?,Ngo,"Santiago, , Chile","Santiago, , Chile",88.2,3871336.0,"Santiago, Santiago Metropolitan, Chile",70.6,3449741.0,"Santiago, Rio Grande Do Sul, Brazil",64.7,3896433.0,"Cartagena, Valparaíso, Chile",x,,,
2,#Oneless,Ngo,"London, , United Kingdom","London, , United Kingdom",91.7,2643743.0,"London, England, United Kingdom",79.2,2657324.0,"Andover, England, United Kingdom",79.2,2655237.0,"Bolton, England, United Kingdom",x,,,
3,(Red),Ngo,"New York City, , United States","New York City, , United States",93.3,5128581.0,"New York City, New York, United States",76.7,3503.0,"New York, United States",76.7,5128481.0,"New City, New York, United States",x,,,


In [0]:
df.sort_values('score_1', inplace=True)
for i in range(num_matches):
    df[f"use_match_{i+1}"] = ''
df[f"use_match_{1}"] = 'x'
df['no_match_found'] = ''


df

,site,score_1,index_1,name_1,score_2,index_2,name_2,score_3,index_3,name_3,use_match_1,use_match_2,use_match_3,no_match_found
id,,,,,,,,,,,,,,
5,"Accra, , Ghana",85.7,2306104,"Accra, Greater Accra, Ghana",66.7,2306079,"Achiaman, Greater Accra, Ghana",64.3,2306119,"Aburi, Eastern, Ghana",x,,,
1,"Moscow, , Russia",87.5,2921,"Moscow, Russia",87.5,524901,"Moscow, Moscow, Russia",68.8,501215,"Rossosh’, Voronezj, Russia",x,,,
4,"Santiago, , Chile",88.2,3871336,"Santiago, Santiago Metropolitan, Chile",70.6,3449741,"Santiago, Rio Grande Do Sul, Brazil",64.7,3896433,"Cartagena, Valparaíso, Chile",x,,,
2,"London, , United Kingdom",91.7,2643743,"London, England, United Kingdom",79.2,2657324,"Andover, England, United Kingdom",79.2,2655237,"Bolton, England, United Kingdom",x,,,
3,"New York City, , United States",93.3,5128581,"New York City, New York, United States",76.7,3503,"New York, United States",76.7,5128481,"New City, New York, United States",x,,,


In [6]:
# pd.read_csv(f"{path}/entity_name_fix.csv")
# df = pd.read_excel(f"{path}/entity_name_fix.xlsx")
# df.en
with open(f"{path}/entity_name_fix.xlsx") as f:
    print(f)

<_io.TextIOWrapper name='/content/drive/My Drive/active/m5362_20sp_data_warehousing/PD4SDG/PD4SDG_new/data sources/entity_name_fix.xlsx' mode='r' encoding='UTF-8'>


In [0]:
## Read and process entity data
entity = pd.read_csv(f"{path}/output/entity.csv")


check_sites = True
look_for_duplicates = True
num_matches = 3

entity_data = formatter(pd.read_excel(f"{path}/raw/entity_data.xlsx"))
missing_idx = (entity_data[['name','type','city','country']]=='').any(axis=1)
if missing_idx.any():
    display(entity_data[missing_idx])
    raise Exception('Missing data in entity_data.xlsx')
entity_data['site'] = cat(entity_data[['city', 'subcountry', 'country']])

if check_sites:
    site = pd.concat([entity_data['site'], project['site']]).drop_duplicates().reset_index(drop=True)
    site_fix, s = text_match(site, world_site, num_matches)
    site_fix.sort_values('score_1', ascending=True, inplace=True)

    for i in range(num_matches):
        entity_data_similarity[f"use_match_{i+1}"] = ''

    write_file(entity_data_similarity, 'entity_data_similarity')


# idx = proj_site_fix['score_1'] < 100
# proj_site_fix = proj_site_fix.loc[idx]

0                  Moscow, , Russia
1          London, , United Kingdom
2    New York City, , United States
3                 Santiago, , Chile
4                    Accra, , Ghana
Name: site, dtype: object

In [0]:
entity_data = formatter(pd.read_excel(f"{path}/raw/entity_data.xlsx"))
missing_idx = (entity_data[['name','type','city','country']]=='').any(axis=1)
if missing_idx.any():
    display(entity_data[missing_idx])
    raise Exception('Missing data in entity_data.xlsx')
entity_data['type']

In [0]:
world_raw = pd.read_json(f"{path}/raw/world-cities.json").set_index('geonameid')
cols = ['city', 'subcountry', 'country']
world = formatter(world_raw) \
    .rename(columns={'name':'city'})

world = world[cols].drop_duplicates().sort_values(cols[::-1])

F = [f"form_{i+1}" for i in range(4)]
for f in F:
    world[f] = ''

def cat(df):
        return df.add(", ").sum(axis=1).str.strip(", ")

df = world.copy()
df[['city', 'subcountry']] = ''
df2 = df.iloc[:2].copy()
df2['country'] = ['Global', 'European Union']
df = df2.append(df)
df[F[0]] = cat(df[['country']])
country_df = df.drop_duplicates().reset_index(drop=True)


df = world.copy()
df['city'] = ''
df[F[0]] = cat(df[['subcountry', 'country']])
df[F[1]] = cat(df[['subcountry']])
subcountry_df = df.drop_duplicates().reset_index(drop=True)
subcountry_df.index += 1000
pd.



df = world.copy()
df[F[0]] = cat(df[['city', 'subcountry', 'country']])
df[F[1]] = cat(df[['city',               'country']])
df[F[2]] = cat(df[['city', 'subcountry'           ]])
df[F[3]] = cat(df[['city'                         ]])
city_df = df.copy()


df = world.copy()
df['city'] = ''
df[F[0]] = cat(df[['subcountry', 'country']])
df[F[1]] = cat(df[['subcountry']])
subcountry_df = df.copy().reset_index(drop=True)
subcountry_df.index += 1000


df = world.copy()
df[['city', 'subcountry']] = ''
df2 = df.iloc[:2].copy()
df2['country'] = ['Global', 'European Union']
df = df2.append(df)
df[F[0]] = cat(df[['country']])
country_df = df.copy().reset_index(drop=True)


world = pd.concat([country_df, subcountry_df]).drop_duplicates().reset_index(drop=True)
world = pd.concat([world, city_df]).drop_duplicates()
world.index.rename('geonameid', inplace=True)

world_site = world[F]

with pd.ExcelWriter(f'{path}/world.xlsx') as writer:  
    world_raw.to_excel(writer, sheet_name='raw')
    world.to_excel(writer, sheet_name='clean')
    world_site.to_excel(writer, sheet_name='sites')

In [0]:
# do this in sql

# name fix to make country names in country dataset compatible with country names in the city dataset
rep =  {
    'Bahamas, The': 'Bahamas',
    'British Virgin Is.': 'British Virgin Islands',
    'Burma': 'Myanmar',
    'Central African Rep.': 'Central African Republic',
    'Congo, Dem. Rep.': 'Democratic Republic Of The Congo',
    'Congo, Repub. Of The': 'Republic Of The Congo',
    "Cote D'Ivoire": 'Ivory Coast',
    'Gambia, The': 'Gambia',
    # 'Gaza Strip':
    'Korea, North': 'North Korea',
    'Korea, South': 'South Korea',
    'Macau': 'Macao',
    'Micronesia, Fed. St.': 'Micronesia',
    'N. Mariana Islands': 'Northern Mariana Islands',
    # 'Netherlands Antilles':
    'Turks And Caicos Is': 'Turks And Caicos Islands',
    'Virgin Islands': 'U.S. Virgin Islands',
    # 'West Bank':
}
country_data_raw = pd.read_csv(f"{path}/raw/countries of the world.csv", decimal=',')
country_data = formatter(country_data_raw)
country_data.index += 1
country_data.index.rename('id', inplace=True)

country_data.to_csv(f"{path}/raw/countries of the world cleaned.csv")

# country_data['country'] = country_data['country'].replace(rep).astype('string')
# country_data.set_index('country', inplace=True)

# # Create "Palestine" to be compatible with city dataset
# def combine(old):
#     x = country_data.loc[old]
#     y = x.iloc[0].copy()
#     y[1:3] = x.iloc[:,1:3].sum().astype(int)
#     y[3] = np.round(y[1] / y[2], 2)
#     y[4:] = np.round(x.iloc[:,4:].mean(), 2)
#     return y

# pal = ['Gaza Strip','West Bank']
# country_data.loc['Palestinian Territory'] = combine(pal)

# with pd.ExcelWriter(f'{path}/country_data.xlsx') as writer:  
#     country_data_raw.to_excel(writer, sheet_name='raw')
#     country_data.to_excel(writer, sheet_name='clean')
country_data.head()

,country,region,population,area (sq. mi.),pop. density (per sq. mi.),coastline (coast/area ratio),net migration,infant mortality (per 1000 births),gdp ($ per capita),literacy (%),phones (per 1000),arable (%),crops (%),other (%),climate,birthrate,deathrate,agriculture,industry,service
id,,,,,,,,,,,,,,,,,,,,
1,Afghanistan,Asia (Ex. Near East),31056997,647500,48.0,0.00,23.06,163.07,700,36.0,3.2,12.13,0.22,87.65,1.0,46.60,20.34,0.380,0.240,0.380
2,Albania,Eastern Europe,3581655,28748,124.6,1.26,-4.93,21.52,4500,86.5,71.2,21.09,4.42,74.49,3.0,15.11,5.22,0.232,0.188,0.579
3,Algeria,Northern Africa,32930091,2381740,13.8,0.04,-0.39,31.00,6000,70.0,78.1,3.22,0.25,96.53,1.0,17.14,4.61,0.101,0.600,0.298
4,American Samoa,Oceania,57794,199,290.4,58.29,-20.71,9.27,8000,97.0,259.5,10.00,15.00,75.00,2.0,22.46,3.27,NaN,NaN,NaN
5,Andorra,Western Europe,71201,468,152.1,0.00,6.60,4.05,19000,100.0,497.2,2.22,0.00,97.78,3.0,8.71,6.25,NaN,NaN,NaN


In [0]:
entity_data['type'].value_counts()

Ngo         3932
Priv        1831
Acad        1110
Subgov       498
Sci          449
Natgov       232
Intergov     196
Un           126
Supgov         1
Name: type, dtype: Int64

In [0]:
## Read and process entity data
look_for_duplicates = True
num_matches = 3

entity_data = formatter(pd.read_excel(f"{path}/raw/entity_data.xlsx"))
missing_idx = (entity_data[['name','type','city','country']]=='').any(axis=1)
if missing_idx.any():
    display(entity_data[missing_idx])
    raise Exception('Missing data in entity_data.xlsx')

if look_for_duplicates:
    A = entity_data['name'][:100]
    B = A
    left, s = text_match(A, B, num_matches+1)
    # display(left.head())
    # idx = left['score_1'] > 0.8
    # left = left.loc[idx]

    attr = ['score', 'index', 'name']
    srt = [f"{s}_{i}" for i in range(1, num_matches+1) for s in attr]
    entity_data_similarity = left[srt].sort_values('score_1', ascending=False)

    entity_data_similarity['keep_orig'] = 'x'
    for i in range(num_matches):
        entity_data_similarity[f"use_match_{i+1}"] = ''

    write_file(entity_data_similarity, 'entity_data_similarity')
# entity_data_similarity.head()
entity_data_similarity

,score_1,index_1,name_1,score_2,index_2,name_2,score_3,index_3,name_3,keep_orig,use_match_1,use_match_2,use_match_3
orig,,,,,,,,,,,,,
Acción International,78.3,89,Actionaid International,75.0,98,Add International,71.4,22,A Rocha International,x,,,
Actionaid International,78.3,63,Acción International,69.6,22,A Rocha International,69.6,98,Add International,x,,,
Abundance Foundation,75.0,48,Abusua Foundation,70.0,41,Abibiman Foundation,48.4,4,Community And Family Foundation,x,,,
Abusua Foundation,75.0,47,Abundance Foundation,73.7,41,Abibiman Foundation,41.9,4,Community And Family Foundation,x,,,
Add International,75.0,63,Acción International,71.4,22,A Rocha International,69.6,89,Actionaid International,x,,,
Abibiman Foundation,73.7,48,Abusua Foundation,70.0,47,Abundance Foundation,42.1,86,Action Fund,x,,,
Aarhus University,72.2,28,Aalborg University,48.0,29,"Aalto University, Finland",35.9,96,Adama Science And Technology University,x,,,
Aalborg University,72.2,30,Aarhus University,52.0,29,"Aalto University, Finland",38.5,96,Adama Science And Technology University,x,,,
A Rocha International,71.4,98,Add International,71.4,63,Acción International,69.6,89,Actionaid International,x,,,


In [0]:
entity_data.head()

# sites = project['site']

NameError: ignored

In [0]:
project['site']


proj_site_fix, s = text_match(project['site'].drop_duplicates(), world_site, num_matches=3)
proj_site_fix.sort_values('score_1', ascending=False, inplace=True)
idx = proj_site_fix['score_1'] < 100
proj_site_fix = proj_site_fix.loc[idx]


,site,score_1,index_1,name_1,score_2,index_2,name_2,score_3,index_3,name_3
2221,Uruaguay,87.5,235,Uruguay,62.5,3980760,"Uruapan, Michoacán, Mexico",62.5,2806,"Aragua, Venezuela"
3396,Kirabati,87.5,114,Kiribati,66.7,550671,"Kharabali, Astrakhan, Russia",62.5,769893,"Kabaty, Masovian Voivodeship, Poland"
2190,Unitedkingdom,85.7,233,United Kingdom,63.2,2656708,"Ayr, Scotland, United Kingdom",60.0,2656173,"Bath, England, United Kingdom"
2443,Phillipines,81.8,172,Philippines,56.2,1679678,"Wao, Autonomous Region In Muslim Mindanao, Phi...",56.2,1711437,"Iba, Central Luzon, Philippines"
2220,Congo,80.0,2427637,"Mongo, Guéra, Chad",66.7,1848277,"Yonago, Tottori, Japan",66.7,4682991,"Conroe, Texas, United States"
2680,Roatan,66.7,1258076,"Rohtak, Haryana, India",66.7,1281019,"Hotan, Xinjiang Uygur Zizhiqu, China",66.7,2511700,"Rota, Andalusia, Spain"
2745,Timor-Leste,54.5,3165185,"Trieste, Friuli Venezia Giulia, Italy",45.5,3359041,"Worcester, Western Cape, South Africa",45.5,2510725,"Tacoronte, Canary Islands, Spain"
2792,Cote D'Ivoire,46.2,7280414,"Cole Harbour, Nova Scotia, Canada",46.2,3023141,"Courbevoie, Île-De-France, France",40.0,4142290,"Dover, Delaware, United States"


In [0]:
world_raw = pd.read_json(f"{path}/raw/world-cities_json.json").set_index('geonameid').drop_duplicates()

cols = ['city', 'subcountry', 'country']
world = formatter(world_raw) \
    .rename(columns={'name':'city'})
world = world[cols].sort_values(cols[::-1])

def cat(df):
        return df.add(", ").sum(axis=1).str.strip(", ")

A = pd.DataFrame()
A[F[0]] = cat(world[['city', 'subcountry', 'country']])
A[F[1]] = cat(world[['city',               'country']])
A[F[2]] = cat(world[['city', 'subcountry'           ]])
A[F[3]] = cat(world[['city'                         ]])

B = pd.DataFrame()
B[F[0]] = cat(world[['subcountry', 'country']])
B[F[1]] = cat(world[[              'country']])

C = pd.DataFrame()
C[F[0]] = cat(world[['country']])

D = C.iloc[:2].copy()
D[F[0]] = ['Global', 'European Union']

world_site = pd.concat([D, C, B]).reset_index(drop=True)
world_site = pd.concat([world_site, A]).drop_duplicates()

world_site.shape

(25880, 4)

In [0]:
world_raw = pd.read_json(f"{path}/raw/world-cities_json.json").set_index('geonameid')
cols = ['city', 'subcountry', 'country']
world = formatter(world_raw) \
    .rename(columns={'name':'city'})
world = world[cols].sort_values(cols[::-1])#['country', 'subcountry', 'city'])

F = [f"form_{i+1}" for i in range(4)]
for f in F:
    world[f] = ''

def cat(df):
        return df.add(", ").sum(axis=1).str.strip(", ")

A = world.copy()
A[F[0]] = cat(A[cols])
A[F[1]] = cat(A[['city', 'country']])
A[F[2]] = cat(A[['city', 'subcountry']])
A[F[3]] = cat(A[['city']])


B = world.copy()
B['city'] = ''
B = B.drop_duplicates()
B[F[0]] = cat(B[['subcountry', 'country']])
B[F[1]] = cat(B[['subcountry']])

C = world.copy()
C[['city', 'subcountry']] = ''
D = C.iloc[:2].copy()
D['country'] = ['Global', 'European Union']
C = D.append(C).drop_duplicates()
C[F[0]] = cat(C[['country']])

D = pd.concat([C, B], ignore_index=True)
world = pd.concat([D, A])
world.index.rename('geonameid', inplace=True)


world.head()
# world.tail()

# A = world.copy()
# A['city'] = ''
# A = A.drop_duplicates()


# B = A.copy()
# B['subcountry'] = ''
# B = B.drop_duplicates()

# C = B.iloc[:2].copy()
# C['country'] = ['Global', 'European Union']

# D = pd.concat([C, B, A], ignore_index=True)
# world = pd.concat([D, world])
# world.index.rename('geonameid', inplace=True)

# with pd.ExcelWriter(f'{path}/world.xlsx') as writer:  
#     world_raw.to_excel(writer, sheet_name='raw')
#     world.to_excel(writer, sheet_name='clean')
#     L = {'site':['city','subcountry','country'],
#         'nocountry':['city','subcountry'],
#         'nosubcountry':['city','country'],
#         'nocity':['subcountry','country'],
#         'city':['city'],
#         'subcountry':['subcountry'],
#         'country':['country'],
#     }
#     world_mini = dict()
#     for lev, cols in L.items():
#         X = world[cols].add(", ").sum(axis=1).str.strip(", ").drop_duplicates().sort_index()
#         X = X[X != '']
#         world_mini[lev] = X
#         X.to_excel(writer, sheet_name=lev)

,city,subcountry,country,form_1,form_2,form_3,form_4
geonameid,,,,,,,
0,,,Global,Global,,,
1,,,European Union,European Union,,,
2,,,Afghanistan,Afghanistan,,,
3,,,Aland Islands,Aland Islands,,,
4,,,Albania,Albania,,,


In [0]:
entity_raw = project_entity_raw.drop(columns=['un_idx', 'n']).drop_duplicates()\
    .sort_values(['name', 'country', 'city']).rename(columns={'name':'name_orig'})

# apply entity_fix
entity_fix = formatter(pd.read_excel(f"{path}/raw/entity_fix.xlsx"))
entity = pd.merge(entity_raw, entity_fix, how='left', on='name_orig')
idx = entity['name'].isna()
entity.loc[idx, 'name'] = entity.loc[idx, 'name_orig']

# merge entity_data
entity = pd.merge(entity, entity_data, how='left', on='name', suffixes=('_orig',''))

# look for unmatched entities
idx = entity['type'].isna()
if idx.any():
    entity_nomatch = entity.loc[idx,:'country_orig'].drop_duplicates()

    num_matches = 3
    A = entity_nomatch['name_orig']
    B = entity_data['name']
    left, s = text_match(A, B, num_matches)

    attr = ['type', 'city', 'subcountry', 'country']
    for i in range(num_matches):
        attr_new = [f"{s}_{i}" for s in attr]
        right = entity_data[attr].rename(columns=dict(zip(attr, attr_new)))
        left = left.join(right, on=f"index_{i}", how='left')

    attr = ['score', 'index', 'name'] + attr
    srt = [f"{s}_{i}"   for i in range(num_matches) for s in attr]
    ren = [f"{s}_{i+1}" for i in range(num_matches) for s in attr]
    entity_nomatch_similarity = left[srt].rename(columns=dict(zip(srt,ren))).sort_values('score_1', ascending=False)

    entity_nomatch_similarity['use_orig'] = ''
    for i in range(num_matches):
        entity_nomatch_similarity[f'use_match_{i+1}'] = ''

    for s in attr[2:]:
        entity_nomatch_similarity[f'{s}_new'] = ''

write_file(entity_nomatch_similarity, 'entity_nomatch_similarity')
entity_nomatch_similarity.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Support for multi-dimensional indexing (e.g. `index[:, None]`) on an Index is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


,score_1,index_1,name_1,type_1,city_1,subcountry_1,country_1,score_2,index_2,name_2,type_2,city_2,subcountry_2,country_2,score_3,index_3,name_3,type_3,city_3,subcountry_3,country_3,use_orig,use_match_1,use_match_2,use_match_3,name_new,type_new,city_new,subcountry_new,country_new
orig,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"South-South Gate,",58.8,6813,South West Water,Private Sector,Exeter,,United Kingdom,52.9,6832,Southern Water,Private Sector,Worthing,,United Kingdom,48.0,6810,South Staffordshire Water,Private Sector,Walsall,,United Kingdom,,,,,,,,,
#Name?,42.9,746,Banamex,Private Sector,Mexico City,,Mexico,37.5,1871,Cyclamed,Ngo,Boulogne-Billancourt,,France,37.5,4710,Metameta,Ngo,Hertogenbosch,,Netherlands,,,,,,,,,


In [0]:
world_raw = pd.read_json(f"{path}/raw/world-cities_json.json").set_index('geonameid')
world = formatter(world_raw) \
    .rename(columns={'name':'city'}) \
    [['city', 'subcountry', 'country']] \
    .sort_values(['country', 'subcountry', 'city']) \

F = [f"form_{i+1}" for i in range(4)]
world[F] = ''

L = {'site':['city','subcountry','country'],
    'nocountry':['city','subcountry'],
    'nosubcountry':['city','country'],
    'nocity':['subcountry','country'],
    'city':['city'],
    'subcountry':['subcountry'],
    'country':['country'],
}
D = dict()
for lev, cols in L.items():
    X = world[cols].add(", ").sum(axis=1).str.strip(", ").drop_duplicates()#.sort_index()
    X = X[X != '']
    d_mini[lev] = X
world = pd.concat([d for  in world_mini.values()]).drop_duplicates()
    
    
    X.to_excel(writer, sheet_name=lev)


with pd.ExcelWriter(f'{path}/world.xlsx') as writer:  
    world_raw.to_excel(writer, sheet_name='raw')
    world.to_excel(writer, sheet_name='clean')


In [0]:
num_matches = 2
A = project['site'].drop_duplicates()
A = A[A != '']
B = world_mini['site']
left, s = text_match(A, B, num_matches)

project_site_similarity = left
project_site_similarity.head()
# project_site_fix_dict = fix_site(project['site'].to_numpy())
# project_site_fix = pd.DataFrame.from_dict(project_site_fix_dict, orient='index')
# write_file(project_site_fix, 'project_site_fix')

,score_0,index_0,name_0,score_1,index_1,name_1
orig,,,,,,
Albania,100.0,4,Albania,57.1,12,Armenia
Algeria,100.0,5,Algeria,71.4,158,Nigeria
Antigua And Barbuda,100.0,10,Antigua And Barbuda,61.3,354,"Saint John, Antigua And Barbuda"
Argentina,100.0,11,Argentina,66.7,12,Armenia
Aruba,100.0,13,Aruba,60.0,53,Cuba


In [0]:
project_site_similarity

,score_0,index_0,name_0,score_1,index_1,name_1
orig,,,,,,
Albania,100.0,4,Albania,57.1,12,Armenia
Algeria,100.0,5,Algeria,71.4,158,Nigeria
Antigua And Barbuda,100.0,10,Antigua And Barbuda,61.3,354,"Saint John, Antigua And Barbuda"
Argentina,100.0,11,Argentina,66.7,12,Armenia
Aruba,100.0,13,Aruba,60.0,53,Cuba
Australia,100.0,14,Australia,77.8,15,Austria
Austria,100.0,15,Austria,77.8,14,Australia
Bahamas,100.0,17,Bahamas,57.1,168,Panama
Bangladesh,100.0,19,Bangladesh,58.8,454,"Dhaka, Bangladesh"


In [0]:
def fix_site(orig, num_matches=1):
    B = world_mini['site']
    left, s = text_match(orig, B, num_matches)

    
    # display(left.head(10))


    # attr = ['name']
    # for i in range(num_matches):
    #     attr_new = [f"{s}_{i}" for s in attr]
    #     right = entity_data[attr].rename(columns=dict(zip(attr, attr_new)))
    #     left = left.join(right, on=f"index_{i}", how='left')




    # site = {s:{'scores':[0.0 for _ in range(n)], 'matches':[None for _ in range(n)]} for s in orig}
    
    # # iterate over sites
    # for lev, w in world_mini.items():
    #     d = textdist(orig, w)
        
    #     for s, val in site.items():
            

    #     # iterate over known world location in multiple levels like city, subcountry, country and subset of these 3 items
        

    #         # compute distance scores & find best
    #         score = textdist(np.array(w), s)


    #         newus_scorus = score.max()

    #         # if there is a better score than the current best, we record this improved match
    #         if newus_scorus > biggus_scorus:
    #             biggus_scorus = newus_scorus
    #             site[s]['score'] = biggus_scorus
    #             hits = w[score==biggus_scorus].index
    #             n = len(hits)
    #             site[s]['n'] = n
    #             match = world.loc[hits, ['city','subcountry', 'country']]
    #             if lev in ['subcountry', 'country', ' nocity']:
    #                 match['city'] = ''
    #                 if lev in ['country']:
    #                     match['subcountry'] = ''
    #             site[s]['match'] = match.to_dict(orient='records')
    # return site

# project_site_fix_dict = fix_site(project['site'].to_numpy())
# project_site_fix = pd.DataFrame.from_dict(project_site_fix_dict, orient='index')
# write_file(project_site_fix, 'project_site_fix')


project_site_fix_dict = fix_site(project['site'].to_numpy())
project_site_fix = pd.DataFrame.from_dict(project_site_fix_dict, orient='index')
write_file(project_site_fix, 'project_site_fix')

ERROR! Session/line number was not unique in database. History logging moved to new session 63


,score_0,index_0,name_0
orig,,,
,0.0,0,"""Sail Of Hope"""
Albania,100.0,271,Albania
Algeria,100.0,292,Algeria
Antigua And Barbuda,100.0,408,Antigua And Barbuda
Argentina,100.0,463,Argentina
Aruba,100.0,494,Aruba
Australia,100.0,669,Australia
Austria,100.0,686,Austria
Bahamas,100.0,720,Bahamas


ValueError: ignored

In [0]:
entity_raw = project_entity_raw.drop(columns=['un_idx', 'n']).drop_duplicates().sort_values(['name', 'country', 'city'])
# entity_raw['subcountry'] = ''
entity_raw = entity_raw[['name','type','city','country']].rename(columns={'name':'name_orig'})



entity_fix = formatter(pd.read_excel(f"{path}/raw/entity_fix.xlsx"))
entity_data = formatter(pd.read_excel(f"{path}/raw/entity_data.xlsx"))
missing_idx = (entity_data[['name','type','city','country']]=='').any(axis=1)
if missing_idx.any():
    display(entity_data[missing_idx])
    raise Exception('Missing data in entity_data.xlsx')


entity = pd.merge(entity_raw, entity_fix, how='left', on='name_orig')

idx = entity['name'].isna()
entity.loc[idx, 'name'] = entity.loc[idx, 'name_orig']

entity = pd.merge(entity, entity_data, how='left', on='name', suffixes=('_orig',''))

idx = entity['type'].isna()
entity_nomatch = entity.loc[idx,:'country_orig'].drop_duplicates()

num_matches=3
fix, _ = text_match(entity_nomatch['name_orig'], entity_data['name'], num_matches)
for i in range(num_matches):
    fix = fix.join(entity_data.set_index('name'), how='left', on=f"name_{i}")
    L = ['type', 'city', 'subcountry', 'country']
    fix.rename(columns={c:f"{c}_{i}" for c in L}, inplace=True)

L = ['score', 'name'] + L
fix = fix[[f"{attr}_{i}" for i in range(num_matches) for attr in L]]
fix.sort_values('score_0', ascending=False, inplace=True)

entity_nomatch_fix = entity_nomatch.join(fix, on='name_orig', how='left').sort_values('score_0', ascending=False)


entity_nomatch_fix['use_orig'] = ''
for i in range(num_matches):
    entity_nomatch_fix[f'use_match_{i}'] = ''

for c in L[1:]:
    entity_nomatch_fix[f'{c}_new'] = ''

write_file(entity_nomatch_fix, 'entity_nomatch_fix')
entity_nomatch_fix

,name_orig,type_orig,city_orig,subcountry_orig,country_orig,score_0,name_0,type_0,city_0,subcountry_0,country_0,score_1,name_1,type_1,city_1,subcountry_1,country_1,score_2,name_2,type_2,city_2,subcountry_2,country_2,use_orig,use_match_0,use_match_1,use_match_2,name_new,type_new,city_new,subcountry_new,country_new
6136,New World Hope Organization,Ngo,Wah Cantt,,Pakistan,0.962963,New World Hope Organization,Ngo,Wah Cantt,,Pakistan,0.666667,World Trade Organization,Intergovernmental Organization,Geneva,,Switzerland,0.629630,World Health Organization,United Nations Entity,Geneva,,Switzerland,,,,,,,,,
6134,New World Hope Organization (Nwho),,,,,0.794118,New World Hope Organization,Ngo,Wah Cantt,,Pakistan,0.558824,World Trade Organization,Intergovernmental Organization,Geneva,,Switzerland,0.529412,World Health Organization,United Nations Entity,Geneva,,Switzerland,,,,,,,,,
8109,"South-South Gate,",,,,,0.588235,South West Water,Private Sector,Exeter,,United Kingdom,0.529412,Southern Water,Private Sector,Worthing,,United Kingdom,0.480000,South Staffordshire Water,Private Sector,Walsall,,United Kingdom,,,,,,,,,
1,#Name?,Ngo,Santiago,,Chile,0.428571,Banamex,Private Sector,Mexico City,,Mexico,0.375000,Cyclamed,Ngo,Boulogne-Billancourt,,France,0.375000,Metameta,Ngo,Hertogenbosch,,Netherlands,,,,,,,,,


In [0]:
entity_raw = project_entity_raw.drop(columns=['un_idx', 'n']).drop_duplicates()\
    .sort_values(['name', 'country', 'city']).rename(columns={'name':'name_orig'})

# apply entity_fix
entity_fix = formatter(pd.read_excel(f"{path}/raw/entity_fix.xlsx"))
entity = pd.merge(entity_raw, entity_fix, how='left', on='name_orig')


idx = entity['name'].isna()
entity.loc[idx, 'name'] = entity.loc[idx, 'name_orig']
entity = pd.merge(entity, entity_data, how='left', on='name', suffixes=('_orig',''))


entity.iloc[6120:6140]

,name_orig,type_orig,city_orig,country_orig,name,type,city,subcountry,country
6120,New Economics Foundation,Ngo,London,United Kingdom,New Economics Foundation,Ngo,London,,United Kingdom
6121,New Energy And Industrial Technology Developme...,,,,Japan,National Government,Toyko,,Japan
6122,New Energy And Industrial Technology Developme...,,,,Japan,National Government,Toyko,,Japan
6123,New England Aquarium,Scientific Community,Boston,United States,New England Aquarium,Scientific Community,Boston,,United States
6124,New England Seafood,Private Sector,Chessington,United Kingdom,New England Seafoods International Ltd,Private Sector,Chessington,,United Kingdom
6125,New England Seafoods International Ltd,Private Sector,Chessington,United Kingdom,New England Seafoods International Ltd,Private Sector,Chessington,,United Kingdom
6126,"New Horizons,",Ngo,Moscow,Russia,New Horizons,Ngo,Moscow,,Russia
6127,New Jersey,Subnational Government,Trenton,United States,New Jersey,Subnational Government,Trenton,,United States
6128,New Nation Builders Of Leadership And Entrepre...,Ngo,Abuja,Nigeria,New Nation Builders Of Leadership And Entrepre...,Ngo,Abuja,,Nigeria
6129,New Phytologist Trust (United Kingdom Of Great...,,,,New Phytologist Trust,Ngo,Aberystwyth,,United Kingdom


In [0]:
entity_fix.tail()

,name_orig,name
4102,Women Farmers Advancement Network,Women Farmers Advancement Network
4103,New World Hope Organization,New World Hope Organization
4104,Circumpolar Biodiversity Monitoring Program,Circumpolar Biodiversity Monitoring Program
4105,'We Are The Ocean',We Are The Ocean'
4106,New World Hope Organization (Nwho),New World Hope Organization


In [0]:
entity

Output hidden; open in https://colab.research.google.com to view.

In [0]:
entity_nomatch_fix = entity_nomatch.join(fix, on='name_orig', how='left').sort_values('score_0', ascending=False)
entity_nomatch_fix['use_orig'] = ''
for i in range(num_matches):
    entity_nomatch_fix[f'use_match_{i}'] = ''

for c in L[1:]:
    entity_nomatch_fix[f'{c}_new'] = ''

write_file(entity_nomatch_fix, 'entity_nomatch_fix')
entity_nomatch_fix

,name_orig,type_orig,city_orig,subcountry_orig,country_orig,score_0,name_0,type_0,city_0,subcountry_0,country_0,score_1,name_1,type_1,city_1,subcountry_1,country_1,score_2,name_2,type_2,city_2,subcountry_2,country_2,use_orig,use_match_0,use_match_1,use_match_2,name_new,type_new,city_new,subcountry_new,country_new
3,'Nuestra Señora Del Pilar' Primary School,Academic Institution,Monreal Del Campo,,Spain,0.975610,Nuestra Señora Del Pilar' Primary School,Academic Institution,Monreal Del Campo,,Spain,0.390244,National Polytechnic School,Academic Institution,Quito,,Ecuador,0.370370,National Association Of Proprietors Of Private...,Ngo,Abuja,,Nigeria,,,,,,,,,
100,"Acb - Réseau Des Bretons D'Influence,",,,,,0.972973,Acb - Réseau Des Bretons D'Influence,Ngo,Paris,,France,0.326923,Association For Research And Development Of Sc...,Academic Institution,Lisbon,,Portugal,0.324324,Siyah Yisrael Bet Knesset,Ngo,Abuja,,Nigeria,,,,,,,,,
9986,Women Farmers Advancement Network,Ngo,Kano,,Nigeria,0.969697,Women Farmers Advancement Network,Ngo,Kano,,Nigeria,0.454545,Fresh Water Action Network,Ngo,London,,United Kingdom,0.454545,Water Integrity Network,Ngo,Berlin,,Germany,,,,,,,,,
6135,New World Hope Organization,Ngo,Wah Cantt,,Pakistan,0.962963,New World Hope Organization,Ngo,Wah Cantt,,Pakistan,0.666667,World Trade Organization,Intergovernmental Organization,Geneva,,Switzerland,0.629630,World Health Organization,United Nations Entity,Geneva,,Switzerland,,,,,,,,,
1773,Circumpolar Biodiversity Monitoring Program,Ngo,Akureyri,,Iceland,0.953488,Circumpolar Biodiversity Monitoring Program,Ngo,Akureyri,,Iceland,0.407407,United Nations University Fisheries Training P...,United Nations Entity,Reykjavik,,Iceland,0.395349,National Biodiversity Network,Ngo,Nottingham,,United Kingdom,,,,,,,,,
4,'We Are The Ocean',Ngo,Flushing,,United States,0.944444,We Are The Ocean',Ngo,Flushing,,United States,0.888889,We Are The Oceans,Ngo,London,,United Kingdom,0.666667,We Are The Present,Ngo,Lima,,Peru,,,,,,,,,
6134,New World Hope Organization (Nwho),,,,,0.794118,New World Hope Organization,Ngo,Wah Cantt,,Pakistan,0.558824,World Trade Organization,Intergovernmental Organization,Geneva,,Switzerland,0.529412,World Health Organization,United Nations Entity,Geneva,,Switzerland,,,,,,,,,
8108,"South-South Gate,",,,,,0.588235,South West Water,Private Sector,Exeter,,United Kingdom,0.529412,Southern Water,Private Sector,Worthing,,United Kingdom,0.480000,South Staffordshire Water,Private Sector,Walsall,,United Kingdom,,,,,,,,,
1,#Name?,Ngo,Santiago,,Chile,0.428571,Banamex,Private Sector,Mexico City,,Mexico,0.375000,Cyclamed,Ngo,Boulogne-Billancourt,,France,0.375000,Metameta,Ngo,Hertogenbosch,,Netherlands,,,,,,,,,


In [0]:
entity_nomatch.head()

,name_orig,type_orig,city_orig,subcountry_orig,country_orig
1,#Name?,Ngo,Santiago,,Chile
3,'Nuestra Señora Del Pilar' Primary School,Academic Institution,Monreal Del Campo,,Spain
4,'We Are The Ocean',Ngo,Flushing,,United States
100,"Acb - Réseau Des Bretons D'Influence,",,,,
1773,Circumpolar Biodiversity Monitoring Program,Ngo,Akureyri,,Iceland


In [0]:
entity

Output hidden; open in https://colab.research.google.com to view.

In [0]:
# entity_nomatch.join(df, on='name_orig').sort_values('score_0', ascending=False)
num_matches=3
i = 0
X = df
for i in range(num_matches):
    X = pd.merge(X, entity_data, left_on=f"name_{i}", right_on='name').drop(columns=['name'])
    L = ['type', 'city', 'subcountry', 'country']
    X.rename(columns={c:f"{c}_{i}" for c in L}, inplace=True)

X = X[[f"{attr}_{i}" for i in range(num_matches) for attr in ['score', 'name', 'type', 'city', 'subcountry', 'country']]]
X
# for i in range(num_matches):
#     X = 

,score_0,name_0,type_0,type_0,city_0,city_0,subcountry_0,subcountry_0,country_0,country_0,score_1,name_1,type_1,type_1,city_1,city_1,subcountry_1,subcountry_1,country_1,country_1,score_2,name_2,type_2,type_2,city_2,city_2,subcountry_2,subcountry_2,country_2,country_2
0,0.975610,Nuestra Señora Del Pilar' Primary School,Academic Institution,Academic Institution,Monreal Del Campo,Monreal Del Campo,,,Spain,Spain,0.390244,National Polytechnic School,Academic Institution,Academic Institution,Quito,Quito,,,Ecuador,Ecuador,0.370370,National Association Of Proprietors Of Private...,Ngo,Ngo,Abuja,Abuja,,,Nigeria,Nigeria
1,0.972973,Acb - Réseau Des Bretons D'Influence,Ngo,Ngo,Paris,Paris,,,France,France,0.326923,Association For Research And Development Of Sc...,Academic Institution,Academic Institution,Lisbon,Lisbon,,,Portugal,Portugal,0.324324,Siyah Yisrael Bet Knesset,Ngo,Ngo,Abuja,Abuja,,,Nigeria,Nigeria
2,0.969697,Women Farmers Advancement Network,Ngo,Ngo,Kano,Kano,,,Nigeria,Nigeria,0.454545,Fresh Water Action Network,Ngo,Ngo,London,London,,,United Kingdom,United Kingdom,0.454545,Water Integrity Network,Ngo,Ngo,Berlin,Berlin,,,Germany,Germany
3,0.962963,New World Hope Organization,Ngo,Ngo,Wah Cantt,Wah Cantt,,,Pakistan,Pakistan,0.666667,World Trade Organization,Intergovernmental Organization,Intergovernmental Organization,Geneva,Geneva,,,Switzerland,Switzerland,0.629630,World Health Organization,United Nations Entity,United Nations Entity,Geneva,Geneva,,,Switzerland,Switzerland
4,0.794118,New World Hope Organization,Ngo,Ngo,Wah Cantt,Wah Cantt,,,Pakistan,Pakistan,0.558824,World Trade Organization,Intergovernmental Organization,Intergovernmental Organization,Geneva,Geneva,,,Switzerland,Switzerland,0.529412,World Health Organization,United Nations Entity,United Nations Entity,Geneva,Geneva,,,Switzerland,Switzerland
5,0.953488,Circumpolar Biodiversity Monitoring Program,Ngo,Ngo,Akureyri,Akureyri,,,Iceland,Iceland,0.407407,United Nations University Fisheries Training P...,United Nations Entity,United Nations Entity,Reykjavik,Reykjavik,,,Iceland,Iceland,0.395349,National Biodiversity Network,Ngo,Ngo,Nottingham,Nottingham,,,United Kingdom,United Kingdom
6,0.944444,We Are The Ocean',Ngo,Ngo,Flushing,Flushing,,,United States,United States,0.888889,We Are The Oceans,Ngo,Ngo,London,London,,,United Kingdom,United Kingdom,0.666667,We Are The Present,Ngo,Ngo,Lima,Lima,,,Peru,Peru
7,0.588235,South West Water,Private Sector,Private Sector,Exeter,Exeter,,,United Kingdom,United Kingdom,0.529412,Southern Water,Private Sector,Private Sector,Worthing,Worthing,,,United Kingdom,United Kingdom,0.480000,South Staffordshire Water,Private Sector,Private Sector,Walsall,Walsall,,,United Kingdom,United Kingdom
8,0.428571,Banamex,Private Sector,Private Sector,Mexico City,Mexico City,,,Mexico,Mexico,0.375000,Cyclamed,Ngo,Ngo,Boulogne-Billancourt,Boulogne-Billancourt,,,France,France,0.375000,Metameta,Ngo,Ngo,Hertogenbosch,Hertogenbosch,,,Netherlands,Netherlands


In [0]:
def fix_site(orig):
    # uniquify to avoid wasted effort
    orig = np.unique(orig).tolist()

    # remove trivial case if present
    try:
        orig.remove('')
    except:
        pass

    world['site']

    # site dict records original, best current score, number of ties for that score, and matches with that score




    site = {s:{'score':0.0, 'n':0, 'match':[]} for s in orig}

    # chosen distance metric
    
    
    # iterate over sites
    for s, val in site.items():
        # best current score
        biggus_scorus = 0.0

        # iterate over known world location in multiple levels like city, subcountry, country and subset of these 3 items
        for lev, w in world_mini.items():

            # compute distance scores & find best
            score = textdist(w.to_numpy(), s)
            newus_scorus = score.max()

            # if there is a better score than the current best, we record this improved match
            if newus_scorus > biggus_scorus:
                biggus_scorus = newus_scorus
                site[s]['score'] = biggus_scorus
                hits = w[score==biggus_scorus].index
                n = len(hits)
                site[s]['n'] = n
                match = world.loc[hits, ['city','subcountry', 'country']]
                if lev in ['subcountry', 'country', ' nocity']:
                    match['city'] = ''
                    if lev in ['country']:
                        match['subcountry'] = ''
                site[s]['match'] = match.to_dict(orient='records')
    return site

project_site_fix_dict = fix_site(project['site'].to_numpy())
project_site_fix = pd.DataFrame.from_dict(project_site_fix_dict, orient='index')
write_file(project_site_fix, 'project_site_fix')

In [0]:
def fix_site(orig):
    # uniquify to avoid wasted effort
    orig = np.unique(orig).tolist()

    # remove trivial case if present
    try:
        orig.remove('')
    except:
        pass

    # site dict records original, best current score, number of ties for that score, and matches with that score
    site = {s:{'score':0.0, 'n':0, 'match':[]} for s in orig}

    # chosen distance metric
    
    
    # iterate over sites
    for s, val in site.items():
        # best current score
        biggus_scorus = 0.0

        # iterate over known world location in multiple levels like city, subcountry, country and subset of these 3 items
        for lev, w in world_mini.items():

            # compute distance scores & find best
            score = textdist(w.to_numpy(), s)
            newus_scorus = score.max()

            # if there is a better score than the current best, we record this improved match
            if newus_scorus > biggus_scorus:
                biggus_scorus = newus_scorus
                site[s]['score'] = biggus_scorus
                hits = w[score==biggus_scorus].index
                n = len(hits)
                site[s]['n'] = n
                match = world.loc[hits, ['city','subcountry', 'country']]
                if lev in ['subcountry', 'country', ' nocity']:
                    match['city'] = ''
                    if lev in ['country']:
                        match['subcountry'] = ''
                site[s]['match'] = match.to_dict(orient='records')
    return site

project_site_fix_dict = fix_site(project['site'].to_numpy())
project_site_fix = pd.DataFrame.from_dict(project_site_fix_dict, orient='index')
write_file(project_site_fix, 'project_site_fix')

In [0]:
%%time
import xarray as xr
import textdistance
f = np.vectorize(textdistance.levenshtein.normalized_similarity)

site = project['site'].drop_duplicates().tolist()[:10]
site.remove('')

coords = {'site': site,
          'geonameid': world.index,
          'part': world.columns}
data = xr.DataArray(0.0,
                    dims=coords.keys(),
                    coords=coords)

for part, ser in world.iteritems():
    for geonameid, name in ser.iteritems():
        print(part, name)
        data.loc[:, geonameid, part] = f(site, name)
M = data.max(dim=['geonameid','part'])
match = data >= M


d = dict()
for raw in site:
    mask = match.loc[raw].to_pandas()
    d[raw] = world.where(mask).stack()#to_numpy().ravel()

for raw, match in d.items():
    print(raw)
    print(match)
    print()
    print()

Streaming output truncated to the last 5000 lines.
4192205    country    United States
4192289    country    United States
4192375    country    United States
4192674    country    United States
4193699    country    United States
4194474    country    United States
4195701    country    United States
4196586    country    United States
4198322    country    United States
4200671    country    United States
6331909    country    United States
4203696    country    United States
4204007    country    United States
4204230    country    United States
4205196    country    United States
4205885    country    United States
4207226    country    United States
4207400    country    United States
4207783    country    United States
4207981    country    United States
4208442    country    United States
4209448    country    United States
6331908    country    United States
4212684    country    United States
4212992    country    United States
4212995    country    United States
4215110    co

In [0]:
project['site'] = project['loc']


site = project['site'].drop_duplicates().to_frame().set_index('site').drop('')[:10]
site['best_score'] = 0.0
site['best_matches'] = [[] for _ in site.iterrows()]

,best_score,best_matches
site,,
Global,0.0,[]
United States,0.0,[]
United Kingdom,0.0,[]
Sierra Leone,0.0,[]
Fiji,0.0,[]
Samoa,0.0,[]
Nicaragua,0.0,[]
Sri Lanka,0.0,[]
India,0.0,[]


In [0]:
site

,score,match,"(Global, score)","(United States, score)","(United Kingdom, score)","(Sierra Leone, score)","(Fiji, score)","(Samoa, score)","(Nicaragua, score)","(Sri Lanka, score)","(India, score)","(Zimbabwe, score)"
site,,,,,,,,,,,,
Global,0.0,[],0.285714,0.541667,0.518519,0.342857,0.210526,0.25,0.428571,0.391304,0.294118,0.333333
United States,0.0,[],0.285714,0.541667,0.518519,0.342857,0.210526,0.25,0.428571,0.391304,0.294118,0.333333
United Kingdom,0.0,[],0.285714,0.541667,0.518519,0.342857,0.210526,0.25,0.428571,0.391304,0.294118,0.333333
Sierra Leone,0.0,[],0.285714,0.541667,0.518519,0.342857,0.210526,0.25,0.428571,0.391304,0.294118,0.333333
Fiji,0.0,[],0.285714,0.541667,0.518519,0.342857,0.210526,0.25,0.428571,0.391304,0.294118,0.333333
Samoa,0.0,[],0.285714,0.541667,0.518519,0.342857,0.210526,0.25,0.428571,0.391304,0.294118,0.333333
Nicaragua,0.0,[],0.285714,0.541667,0.518519,0.342857,0.210526,0.25,0.428571,0.391304,0.294118,0.333333
Sri Lanka,0.0,[],0.285714,0.541667,0.518519,0.342857,0.210526,0.25,0.428571,0.391304,0.294118,0.333333
India,0.0,[],0.285714,0.541667,0.518519,0.342857,0.210526,0.25,0.428571,0.391304,0.294118,0.333333


In [0]:
world = formatter(pd.read_json(f"{path}/raw/world-cities_json.json")) \
    .rename(columns={'name':'city'}) \
    [['geonameid', 'city', 'subcountry', 'country']].set_index('geonameid') \
    .sort_values(['country', 'subcountry', 'city'])
# world['nocountry'] =    world['city'] + ', ' + world['subcountry']
# world['nosubcountry'] = world['city'] + ', '                              + world['country']
# world['nocity'] =                              world['subcountry'] + ', ' + world['country']
# world['all'] =          world['city'] + ', ' + world['subcountry'] + ', ' + world['country']

world.head(10)

,city,subcountry,country
geonameid,,,
1142170,Fayzabad,Badakhshan,Afghanistan
1141089,Ghormach,Badghis,Afghanistan
1147290,Balkh,Balkh,Afghanistan
1136575,Khulm,Balkh,Afghanistan
1133616,Mazār-E Sharīf,Balkh,Afghanistan
1147242,Bāmyān,Bāmīān,Afghanistan
1142264,Farah,Farah,Afghanistan
1148658,Andkhōy,Faryab,Afghanistan
1133453,Maymana,Faryab,Afghanistan


In [0]:
1+1

2

In [0]:
%%time

import textdistance
f = np.vectorize(textdistance.levenshtein.normalized_similarity)

# Make copy of index from country_raw
X = project[['loc']].drop_duplicates().set_index('loc').drop('').sort_index()

for x, _ in X.iterrows():
    print(x)
    

Albania
Algeria
Antigua And Barbuda
Argentina
Aruba
Australia
Austria
Bahamas
Bangladesh
Belgium
Belize
Benin
Brazil
Brussels
Burundi
California
Cambodia
Cameroon
Canada
Cape Verde
Chile
China
Colombia
Congo
Cook Islands
Costa Rica
Cote D'Ivoire
Crete
Cuba
Curacao
Cyprus
Denmark
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Estonia
Ethiopia
European Union
Fiji
France
French Polynesia
Gambia
Germany
Ghana
Global
Greece
Grenada
Guatemala
Honduras
Iceland
India
Indonesia
Iran
Ireland
Israel
Italy
Jamaica
Japan
Kenya
Kirabati
Kiribati
Lebanon
Madagascar
Malaysia
Maldives
Malta
Marshall Islands
Mauritius
Mexico
Micronesia
Monaco
Montenegro
Montserrat
Mozambique
Myanmar
Nauru
Nepal
Netherlands
New Caledonia
New Zealand
Nicaragua
Nigeria
Niue
Norway
Ottawa
Pakistan
Panama
Papua New Guinea
Peru
Philippines
Phillipines
Portugal
Roatan
Romania
Russia
Saint Kitts And Nevis
Samoa
Sao Tome And Principe
Senegal
Seychelles
Sierra Leone
Singapore
Solomon Islands
South Africa
South Korea
Spain


In [0]:

for idx, x in X.iterrows():




for idx, y in world.iterrows():
    print(y['country'])


    X[y['all']] = f(X.index, y['all'])



# country_match.shape
match = X.idxmax(axis=1).to_frame().reset_index()
# type(match)
# match['hit'] = match.iloc[:,0] == match.iloc[:,1]
# idx = ~match['hit']
# match[idx]
match.head()

In [0]:
%%time

import textdistance
f = np.vectorize(textdistance.levenshtein.normalized_similarity)

# Make copy of index from country_raw
X = project[['loc']].drop_duplicates().set_index('loc').drop('').sort_index()


# for y in world['country'].drop_duplicates():
#     X[y] = f(X.index, y)

for idx, x in X.iterrows():




for idx, y in world.iterrows():
    print(y['country'])


    X[y['all']] = f(X.index, y['all'])



# country_match.shape
match = X.idxmax(axis=1).to_frame().reset_index()
# type(match)
# match['hit'] = match.iloc[:,0] == match.iloc[:,1]
# idx = ~match['hit']
# match[idx]
match.head()

Streaming output truncated to the last 5000 lines.
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Switzerland
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Syria
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Taiwan
Tajikistan

In [0]:
match.head(100)

,loc,0
0,Albania,"Fier, Fier, Albania"
1,Algeria,"Mila, Mila, Algeria"
2,Antigua And Barbuda,"Saint John’S, Saint John, Antigua And Barbuda"
3,Argentina,"Salta, Salta, Argentina"
4,Aruba,"Babijn, N/A, Aruba"
5,Australia,"Kew, Victoria, Australia"
6,Austria,"Graz, Styria, Austria"
7,Bahamas,"Ati, Batha, Chad"
8,Bangladesh,"Dhaka, Dhaka, Bangladesh"
9,Belgium,"Mol, Flanders, Belgium"


In [0]:
world['nosubcountry'].drop_duplicates().shape

(22451,)

In [0]:
import textdistance
f = np.vectorize(textdistance.levenshtein.normalized_similarity)

# Make copy of index from country_raw
country_match = country_raw[[]].copy()
country_match.head()

for y in country_world:
    country_match[y] = f(country_match.index, y)
country_match.shape
country_match.idxmax(axis=1)



Afghanistan                                              Afghanistan
Albania                                                      Albania
Alegre                                                       Algeria
Algeria                                                      Algeria
American Samoa                                        American Samoa
Andorra                                                      Andorra
Angola                                                        Angola
Antigua And Barbuda                              Antigua And Barbuda
Apia                                                         Albania
Argenitina                                                 Argentina
Argentina                                                  Argentina
Armenia                                                      Armenia
Aruba                                                          Aruba
Austraila                                                  Australia
Australia                         

In [0]:

# city_all = pd.read_json(f"{path}/raw/world-cities_json.json", dtype=str)\
#     .apply(lambda x: x.str.strip().str.title()).fillna('').replace(regex=r'^.{0,1}$', value='')\
#     .convert_dtypes()

# country_raw = pd.concat([project['loc'], project_entity_raw['country']]).value_counts().drop('').sort_index()

# %timeit country_all = city_all['country'].unique()
# %timeit country_all = city_all['country'].drop_duplicates()
country_all = city_all['country'].str.strip().drop_duplicates().sort_values()
# type(country_all)
country_all

15                                        Afghanistan
575                                     Aland Islands
65                                            Albania
5460                                          Algeria
332                                    American Samoa
0                                             Andorra
102                                            Angola
64                                           Anguilla
63                                Antigua And Barbuda
129                                         Argentina
85                                            Armenia
572                                             Aruba
368                                         Australia
333                                           Austria
576                                        Azerbaijan
2304                                          Bahamas
1023                                          Bahrain
654                                        Bangladesh
653                         

In [0]:
city.dtypes

country       string
geonameid     string
name          string
subcountry    string
dtype: object

In [0]:
# ! pip install textdistance


     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 26.5MB/s 
     |████████████████████████████████| 143kB 49.1MB/s 
  Created wheel for pyxDamerauLevenshtein: filename=pyxDamerauLevenshtein-1.5.3-cp36-cp36m-linux_x86_64.whl size=86821 sha256=81bb0aaaf7441bd7f703a23ff45174109b5e2e950c42c1db8e4c0df67174d53d
  Stored in directory: /root/.cache/pip/wheels/37/66/2c/863e33407d169ea809ce2908e86556df205dcf5edc4bdd10a5
Successfully built pyxDamerauLevenshtein
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144797 sha256=c1b456cd50241492c4bd0e09b8bdb2ee7e869b5ca1457fb588dc189a94ee6e2c
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
  Created wheel for j

In [0]:
# ! pip install leven
# ! pip install StringDist

from leven import levenshtein
levenshtein('quinten','Quintin')


import stringdist
# stringdist.levenshtein(['test', 'gh'], 'testing')

# r = np.vectorize(stringdist.levenshtein)
# r(['bh','quinten'],['Quintin','k'])

import textdistance
textdistance.hamming.normalized_similarity(['test', 'gh'], 'text')



# levenshtein('quinten',['Quintin', 'gth'])

0.25

In [0]:
idx = city['country'] == "United States"

city.loc[idx, 'subcountry'].unique

city.groupby(['name', 'country'])['subcountry'].count().sort_values()

name                                               country                                     
'Ali Sabieh                                        Djibouti                                        1
Pedroso                                            Portugal                                        1
Pedro Meoqui                                       Mexico                                          1
Pedro Leopoldo                                     Brazil                                          1
Pedro Juan Caballero                               Paraguay                                        1
Pedro Ii                                           Brazil                                          1
Pedro Carbo                                        Ecuador                                         1
Pedro Betancourt                                   Cuba                                            1
Pedreira                                           Brazil                                       

In [0]:
entity_type_raw
write_file(entity_type_raw, '/raw/entity_type_raw')

In [0]:
city = project_entity.groupby(['city', 'country'])['un_idx'].count().drop(['',''])

country = pd.concat([project['loc'], project_entity['country']]).value_counts().drop('')
# X


# X = project['loc']
# project_country = X[X != ''].value_counts()

# X = project_entity['country']
# entity_country = X[X != ''].value_counts()

# all_country = entity_country.add(project_country, fill_value=0).astype(int).sort_values()

# X = X[X != '']
# entity_city = X[X != ''].value_counts()
# entity_city
# X.head()

ERROR! Session/line number was not unique in database. History logging moved to new session 65


United States                       3019
United Kingdom                       815
Switzerland                          782
France                               699
Global                               658
Kenya                                421
Italy                                406
Belgium                              393
Australia                            364
India                                343
Germany                              342
Canada                               306
Brazil                               287
Fiji                                 259
Netherlands                          256
Indonesia                            247
Japan                                241
Sweden                               216
China                                213
New Zealand                          206
Samoa                                206
Honduras                             203
Nigeria                              203
Dominican Republic                   196
Spain           